## Scraping top repositories for GitHub topics using Python

Objectives: 
- 1.Use the requests library to download webpages
- 2.Use beautiful soup to parse and extract information
- 3.Create CSV file with the extracted information

* Step 1: Pick a website and describe your objective. 
   * For this examle we're going to scrape https://github.com/topics
   * From the topics page, we will get a list of topics( the topics are arranged in alphabetical order ).
   * For each topic we'll get the topic title and it's page URL.
   * For each topic we'll get the top 20 repositories in the ti=opic from the topic page( eg: in the 3D topic, we can get the top 20 repositories for that topic. The repositories will have different technologies, username, stars and Repo URL. ( Thus we will grab the username/Repo name, stars and Repo URL.)
   * For each topic we will create a CSV file in the following format:
   
   ```
   Repo Name, UserName, Stars, Repo Url
   three.js, mrdoob, 69700, https://github.com/mrdoob/three.js
   libgdx, libgdx, 18300, https://github.com/libgdx/libgdx
   
   ```

Libraries used are:
- Pandas
- Requests
- BeautifulSoup

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

- We can store the url of the page in a variable. We will then use the requests library to download the page.

In [2]:
url = "https://github.com/topics"

In [3]:
response = requests.get(url)

In [4]:
response.status_code

200

In [5]:
print(len(response.text))

166161


* The above output shows that there are 165726 characters.

You can print the entire text from the page but it is not reccomended as it might slow things down especially in notebooks running on browser tabs

In [6]:
#print(response.text)

* Instesad we can look at the first few lines 

In [7]:
page_content = response.text

In [8]:
page_content[:1000] # This will give the first 1000 lines of HTML for the site

'\n\n<!DOCTYPE html>\n<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"  data-a11y-animated-images="system" data-a11y-link-underlines="false">\n\n\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  \n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-983b05c0927a.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-5d486a4ede8e.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" me

* We can save the HTML of the site and save it locally. 

In [9]:
with open("webpage_sample.html", "w", encoding="utf-8") as f:
    f.write(page_content)

- We can now use beautiful soup to parse and extract  information from the text in the page.

In [10]:
soup = BeautifulSoup(page_content, "html.parser")

In [11]:
type(soup)

bs4.BeautifulSoup

### Lets try grabbing elements from the page.
* This can be done by right clicking any the element and clicking inspect from the options in the browser.

* Let's grab 3D. We can see in the inspect window of the HTML that the 3D is in the p-Tag.
* We can do this by finding all the p-Tags in the page.

In [12]:
p_tags = soup.find_all("p")

In [13]:
len(p_tags)

69

* We observe that there are 69 p-Tags, but if we look at the page, we cannot see 69 topics being displayed on the page currently. This is because we are grabbing more tags than we actually need.
* We can solve this problem by search using the specific class associated with the Tag in question.  

In [14]:
p_tags_topics = soup.find_all("p", class_ = "f3 lh-condensed mb-0 mt-1 Link--primary")
#p_tags_topics = soup.find_all("p", {"class" = "f3 lh-condensed mb-0 mt-1 Link--primary"}) # more generic

In [15]:
len(p_tags_topics)

30

* The 30 p-tags above means that there are probably 30 topics currently displayed on the webpage

In [16]:
p_tags_topics

[<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Algorithm</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amp</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Android</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Angular</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ansible</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">API</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Arduino</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">ASP.NET</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Atom</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Awesome Lists</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amazon Web Services</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Azure</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Babel</p>,
 <p class="f3 lh-condensed m

### Let's now find the coresponding descriptions of the above topics

In [17]:
p_tags_topics_description = soup.find_all("p", class_ = "f5 color-fg-muted mb-0 mt-1")

In [18]:
# Let's print the first 5 topic descriptions.
p_tags_topics_description[:5]

[<p class="f5 color-fg-muted mb-0 mt-1">
           3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Ajax is a technique for creating interactive web applications.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Algorithms are self-contained sequences that carry out a variety of tasks.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Amp is a non-blocking concurrency library for PHP.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Android is an operating system built by Google designed for mobile devices.
         </p>]

- Next we get the URL of to the topic page because from the page we will download more information using the link, since the topics have URL to repositories that have the topics selected and more information on the topic other than the description.

In [19]:
topic_urls = soup.find_all("a", class_ = "no-underline flex-1 d-flex flex-column")

In [20]:
len(topic_urls)

30

In [21]:
topic_urls[0]

<a class="no-underline flex-1 d-flex flex-column" href="/topics/3d">
<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>
<p class="f5 color-fg-muted mb-0 mt-1">
          3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.
        </p>
</a>

- We can construct the url to the pages. Lets construct for a topic in the page.

In [22]:
topic0_url = "https://github.com" + topic_urls[0]["href"]
print(topic0_url)

https://github.com/topics/3d


- We can clean up the topics by removing the tags and only obtaining the text.

In [23]:
p_tags_topics[0]

<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>

In [24]:
topic_titles = []

for tag in p_tags_topics:
    title = tag.text
    topic_titles.append(title)
    
    
print(topic_titles)    

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android', 'Angular', 'Ansible', 'API', 'Arduino', 'ASP.NET', 'Atom', 'Awesome Lists', 'Amazon Web Services', 'Azure', 'Babel', 'Bash', 'Bitcoin', 'Bootstrap', 'Bot', 'C', 'Chrome', 'Chrome extension', 'Command line interface', 'Clojure', 'Code quality', 'Code review', 'Compiler', 'Continuous integration', 'COVID-19', 'C++']


> We now have a list of all the titles without the corresponding tags

- Lets do the same for the topic descriptions and the topic URLs

In [25]:
topic_descriptions = []

for tag in p_tags_topics_description:
    title_descs = tag.text
    topic_descriptions.append(title_descs.strip())
    
# The strip method removes any blank spaces in the text    
print(topic_descriptions)    

['3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.', 'Ajax is a technique for creating interactive web applications.', 'Algorithms are self-contained sequences that carry out a variety of tasks.', 'Amp is a non-blocking concurrency library for PHP.', 'Android is an operating system built by Google designed for mobile devices.', 'Angular is an open source web application platform.', 'Ansible is a simple and powerful automation engine.', 'An API (Application Programming Interface) is a collection of protocols and subroutines for building software.', 'Arduino is an open source platform for building electronic devices.', 'ASP.NET is a web framework for building modern web apps and services.', 'Atom is a open source text editor built with web technologies.', 'An awesome list is a list of awesome things curated by the community.', 'Amazon Web Services provides on-demand cloud computing platforms on a subscription basis.', 'Azure is a cloud co

In [26]:
topic_titles_url = []
base_url = "https://github.com"

for tag in topic_urls:
    topic_titles_url.append(base_url + tag["href"])
    
    
topic_titles_url    

['https://github.com/topics/3d',
 'https://github.com/topics/ajax',
 'https://github.com/topics/algorithm',
 'https://github.com/topics/amphp',
 'https://github.com/topics/android',
 'https://github.com/topics/angular',
 'https://github.com/topics/ansible',
 'https://github.com/topics/api',
 'https://github.com/topics/arduino',
 'https://github.com/topics/aspnet',
 'https://github.com/topics/atom',
 'https://github.com/topics/awesome',
 'https://github.com/topics/aws',
 'https://github.com/topics/azure',
 'https://github.com/topics/babel',
 'https://github.com/topics/bash',
 'https://github.com/topics/bitcoin',
 'https://github.com/topics/bootstrap',
 'https://github.com/topics/bot',
 'https://github.com/topics/c',
 'https://github.com/topics/chrome',
 'https://github.com/topics/chrome-extension',
 'https://github.com/topics/cli',
 'https://github.com/topics/clojure',
 'https://github.com/topics/code-quality',
 'https://github.com/topics/code-review',
 'https://github.com/topics/compil

## Let's create a dataframe and save the above data into a CSV file

In [27]:
topics_dictionary = {
    "title": topic_titles,
    "description": topic_descriptions,
    "url": topic_titles_url
}

In [28]:
topics_df = pd.DataFrame(topics_dictionary)

In [29]:
topics_df.head()

,title,description,url
0,3D,3D refers to the use of three-dimensional grap...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android


### Let's save the above dataframe into a CSV file.

In [30]:
topics_df.to_csv("GitHub_Sample.csv", index=None)

# Let's get information from a single topic page URL

In [31]:
topic_page_url = topic_titles_url[0]

In [32]:
topic_page_url

'https://github.com/topics/3d'

In [33]:
response = requests.get(topic_page_url)

In [34]:
response.status_code

200

In [35]:
len(response.text)

477262

In [36]:
soup_topic = BeautifulSoup(response.text, "html.parser")

- Lets now find the information about the first repository which has the Username/Repo Name, the Repository(which is a link) and the number of stars

In [37]:
repository_tags = soup_topic.find_all("h3", class_ = "f3 color-fg-muted text-normal lh-condensed")

In [38]:
len(repository_tags)

20

In [39]:
# These are all the repositories on the page and all the tags.
print(repository_tags)

[<h3 class="f3 color-fg-muted text-normal lh-condensed">
<a class="Link" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
            mrdoob
</a>          /
          <a class="Link text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":576201,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="517d3d5cb9d89752156923904a4238816bc9b51ab7772f3e3644ce897d8dd4e5" data-turbo="false" data-view-component="true" hre

* There are currently 20 repositories in the site from the output above
* Each repository has an a-tag with the username and the repository link

- If we inspect the repository_tag, we will find it has a-tags with username and the repository name(note the repo name has the link to the repository/ project file)

In [40]:
repository_tags[0]

<h3 class="f3 color-fg-muted text-normal lh-condensed">
<a class="Link" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
            mrdoob
</a>          /
          <a class="Link text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":576201,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="517d3d5cb9d89752156923904a4238816bc9b51ab7772f3e3644ce897d8dd4e5" data-turbo="false" data-view-component="true" href

- Let's look at the first repository tag

In [61]:
a_tags_repo0 = repository_tags[1].find_all("a")

In [62]:
a_tags_repo0

[<a class="Link" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":45790596,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="14658fab6217ec4ba70f16dd98006d4334793fae49cc25ce2e1c0bb5a8950006" data-turbo="false" data-view-component="true" href="/pmndrs">
             pmndrs
 </a>,
 <a class="Link text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":172521926,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="629be4efc1260d27fe29201a1901eb808cbf995e4a51d877282b7164242dbadf" data-turbo="false" data-view-component="true" href="/pmndrs/react-three-fiber">
             react-three-fiber
 </a>

> Repository 0 has two "a-tags". Containing the user name and the repository name.

In [60]:
# We can take the first a-tag and extract the user name
a_tags_repo0[0].text.strip()

'mrdoob'

In [44]:
# We can take the first a-tag and extract the repository
a_tags_repo0[1].text.strip()

'three.js'

In [45]:
# We can take the first a-tag and extract the repository URL
base_url = "https://github.com"
repo_url = base_url + a_tags_repo0[1]["href"]

print(repo_url)

https://github.com/mrdoob/three.js


In [46]:
# We can take the number of stars by inspecting the page.
num_of_stars = soup_topic.find_all("span", class_ = "Counter js-social-count")

In [47]:
len(num_of_stars)

20

In [48]:
# These are the tags containing the number of stars( i.e: The rating of the repositories)
print(num_of_stars)

[<span aria-label="94169 users starred this repository" class="Counter js-social-count" data-plural-suffix="users starred this repository" data-singular-suffix="user starred this repository" data-turbo-replace="true" data-view-component="true" id="repo-stars-counter-star" title="94,169">94.2k</span>, <span aria-label="23625 users starred this repository" class="Counter js-social-count" data-plural-suffix="users starred this repository" data-singular-suffix="user starred this repository" data-turbo-replace="true" data-view-component="true" id="repo-stars-counter-star" title="23,625">23.6k</span>, <span aria-label="21849 users starred this repository" class="Counter js-social-count" data-plural-suffix="users starred this repository" data-singular-suffix="user starred this repository" data-turbo-replace="true" data-view-component="true" id="repo-stars-counter-star" title="21,849">21.8k</span>, <span aria-label="21270 users starred this repository" class="Counter js-social-count" data-plur

In [55]:
num_of_stars[1].text.strip()

'23.6k'

> We convert the "k"  into a number from text

In [56]:
def stars_to_number(star_as_str):
    star_as_str = star_as_str.strip()
    if star_as_str[-1] == "k":
        return int(float(star_as_str[:-1]) *1000)
    return star_as_str

In [58]:
stars_to_number(num_of_stars[1].text.strip())

23600

In [68]:
def get_repos_info(h3_tag, stars_tag):
    # this function will get all the repository information.
    # this includes the username, repository name + link and number of stars(rating of repository)
    a_tags = h3_tag.find_all("a")
    username = a_tags[0].text.strip()
    reponame = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]["href"]
    rating = stars_to_number(stars_tag.text.strip())
    return username, reponame, rating, repo_url

> * We will use the above function to abtain the information by passing the Repository_tags and the star_tags

In [69]:
get_repos_info(repository_tags[9], num_of_stars[9])

('metafizzy', 'zdog', 10000, 'https://github.com/metafizzy/zdog')

- Since we have done this for one repository on the page, we do the same for every repository on the page.

In [70]:
len(repository_tags)

20

In [71]:
topic_information_dict = {
    "usernameID": [], 
    "full_repo_name": [], 
    "ratings": [], 
    "project_link": []
}

for i in range(len(repository_tags)):
    full_info = get_repos_info(repository_tags[i], num_of_stars[i])
    
    topic_information_dict["usernameID"].append(full_info[0])
    topic_information_dict["full_repo_name"].append(full_info[1]) 
    topic_information_dict["ratings"].append(full_info[2]) 
    topic_information_dict["project_link"].append(full_info[3]) 

In [72]:
topic_information_dict

{'usernameID': ['mrdoob',
  'pmndrs',
  'libgdx',
  'BabylonJS',
  'ssloy',
  'lettier',
  'aframevr',
  'FreeCAD',
  'CesiumGS',
  'metafizzy',
  'isl-org',
  'blender',
  'timzhang642',
  'a1studmuffin',
  'domlysz',
  'FyroxEngine',
  'nerfstudio-project',
  'google',
  'openscad',
  'spritejs'],
 'full_repo_name': ['three.js',
  'react-three-fiber',
  'libgdx',
  'Babylon.js',
  'tinyrenderer',
  '3d-game-shaders-for-beginners',
  'aframe',
  'FreeCAD',
  'cesium',
  'zdog',
  'Open3D',
  'blender',
  '3D-Machine-Learning',
  'SpaceshipGenerator',
  'BlenderGIS',
  'Fyrox',
  'nerfstudio',
  'model-viewer',
  'openscad',
  'spritejs'],
 'ratings': [94200,
  23600,
  21800,
  21300,
  17700,
  16000,
  15600,
  15000,
  10900,
  10000,
  9400,
  9300,
  9100,
  7400,
  6600,
  6500,
  6400,
  5900,
  5800,
  5200],
 'project_link': ['https://github.com/mrdoob/three.js',
  'https://github.com/pmndrs/react-three-fiber',
  'https://github.com/libgdx/libgdx',
  'https://github.com/Babyl

In [73]:
topic_repos_dataframe = pd.DataFrame(topic_information_dict, index=None)

### These are the top repositories for the topic 3D

In [74]:
topic_repos_dataframe

,usernameID,full_repo_name,ratings,project_link
0,mrdoob,three.js,94200,https://github.com/mrdoob/three.js
1,pmndrs,react-three-fiber,23600,https://github.com/pmndrs/react-three-fiber
2,libgdx,libgdx,21800,https://github.com/libgdx/libgdx
3,BabylonJS,Babylon.js,21300,https://github.com/BabylonJS/Babylon.js
4,ssloy,tinyrenderer,17700,https://github.com/ssloy/tinyrenderer
5,lettier,3d-game-shaders-for-beginners,16000,https://github.com/lettier/3d-game-shaders-for...
6,aframevr,aframe,15600,https://github.com/aframevr/aframe
7,FreeCAD,FreeCAD,15000,https://github.com/FreeCAD/FreeCAD
8,CesiumGS,cesium,10900,https://github.com/CesiumGS/cesium
9,metafizzy,zdog,10000,https://github.com/metafizzy/zdog


- Let's now do the same for all topics not just 3D by creating a single function.

In [89]:
def topics_in_github(topic_url):
    response = requests.get(topic_url)
    if response.status_code != 200:
        raise Exception (f"Failed to load page {topic_url}")
    soup_topic = BeautifulSoup(response.text, "html.parser")
    
    
    
    # We then need the Repo_tags.This contains the Username, Repo_URL and Repo_URL
    repository_tags = soup_topic.find_all("h3", class_ = "f3 color-fg-muted text-normal lh-condensed")
    
    # We then need the Star_tags.This contains the Ratings for this project
    num_of_stars = soup_topic.find_all("span", class_ = "Counter js-social-count")
    
    # We can then find the repository information
    topic_information_dict = {
        "usernameID": [], 
        "full_repo_name": [], 
        "ratings": [], 
        "project_link": []
    }

    for i in range(len(repository_tags)):
        full_info = get_repos_info(repository_tags[i], num_of_stars[i])

        topic_information_dict["usernameID"].append(full_info[0])
        topic_information_dict["full_repo_name"].append(full_info[1]) 
        topic_information_dict["ratings"].append(full_info[2]) 
        topic_information_dict["project_link"].append(full_info[3]) 
        
    return pd.DataFrame(topic_information_dict, index=None)

In [90]:
# From the topics URl we can choose a topic and get information from each using the function above.
topic_titles_url    

['https://github.com/topics/3d',
 'https://github.com/topics/ajax',
 'https://github.com/topics/algorithm',
 'https://github.com/topics/amphp',
 'https://github.com/topics/android',
 'https://github.com/topics/angular',
 'https://github.com/topics/ansible',
 'https://github.com/topics/api',
 'https://github.com/topics/arduino',
 'https://github.com/topics/aspnet',
 'https://github.com/topics/atom',
 'https://github.com/topics/awesome',
 'https://github.com/topics/aws',
 'https://github.com/topics/azure',
 'https://github.com/topics/babel',
 'https://github.com/topics/bash',
 'https://github.com/topics/bitcoin',
 'https://github.com/topics/bootstrap',
 'https://github.com/topics/bot',
 'https://github.com/topics/c',
 'https://github.com/topics/chrome',
 'https://github.com/topics/chrome-extension',
 'https://github.com/topics/cli',
 'https://github.com/topics/clojure',
 'https://github.com/topics/code-quality',
 'https://github.com/topics/code-review',
 'https://github.com/topics/compil

In [91]:
topic_7_url = topic_titles_url[7]

In [92]:
topic_7_url

'https://github.com/topics/api'

In [93]:
topic_7 = topics_in_github(topic_7_url)
topic_7

,usernameID,full_repo_name,ratings,project_link
0,public-apis,public-apis,255000,https://github.com/public-apis/public-apis
1,neovim,neovim,69100,https://github.com/neovim/neovim
2,tiangolo,fastapi,62100,https://github.com/tiangolo/fastapi
3,strapi,strapi,55800,https://github.com/strapi/strapi
4,hoppscotch,hoppscotch,54200,https://github.com/hoppscotch/hoppscotch
5,ocornut,imgui,50000,https://github.com/ocornut/imgui
6,meilisearch,meilisearch,38600,https://github.com/meilisearch/meilisearch
7,slatedocs,slate,35400,https://github.com/slatedocs/slate
8,Kong,insomnia,30500,https://github.com/Kong/insomnia
9,hasura,graphql-engine,30100,https://github.com/hasura/graphql-engine
